<a href="https://colab.research.google.com/github/christophergaughan/sample-work/blob/master/NB_7_GDC_API_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Define Functions to Query and Download Data from the GDC API

## Downloading and Preparing Data

In this section, we are querying the GDC API to retrieve gene expression quantification data for the TCGA-BRCA (breast cancer) project. This involves downloading the necessary files and preparing them for subsequent analysis. Several iterations of this notebook were required to ensure the successful retrieval and processing of the data. Due to this iterative approach, some cells may contain redundant code, which reflects the process of refining and troubleshooting our workflow.

### Step-by-Step Explanation:

1. **Directory Setup**:
   - We start by defining the directory to save the downloaded files. If the directory already exists, we clear it to avoid any conflicts with previous data. Then, we recreate the directory to ensure it is ready for new data.

    ```python
    directory_path = '/content/drive/MyDrive/Colab Notebooks/gdc2_files'
    
    # Clear the directory if it exists
    if os.path.exists(directory_path):
        shutil.rmtree(directory_path)
    
    # Recreate the directory
    os.makedirs(directory_path)
    ```

2. **Querying the GDC API**:
   - We define a function `query_gdc_api` to query the GDC API. The function specifies parameters to filter the data by project ID (TCGA-BRCA), data category (Transcriptome Profiling), and data type (Gene Expression Quantification). The query size is set to 500 to retrieve a substantial number of files.

    ```python
    def query_gdc_api():
        url = "https://api.gdc.cancer.gov/files"

        params = {
            "filters": json.dumps({
                "op": "and",
                "content": [
                    {
                        "op": "in",
                        "content": {
                            "field": "cases.project.project_id",
                            "value": ["TCGA-BRCA"]
                        }
                    },
                    {
                        "op": "in",
                        "content": {
                            "field": "data_category",
                            "value": ["Transcriptome Profiling"]
                        }
                    },
                    {
                        "op": "in",
                        "content": {
                            "field": "data_type",
                            "value": ["Gene Expression Quantification"]
                        }
                    }
                ]
            }),
            "fields": "file_id,file_name,cases.submitter_id",
            "format": "json",
            "size": "500"
        }

        response = requests.get(url, params=params)
        print(f"Query URL: {response.url}")
        if response.status_code == 200:
            data = response.json()
            return data['data']['hits']
        else:
            print(f"Error querying GDC API: {response.status_code}")
            return []
    ```

3. **Retrieving Files**:
   - We call the `query_gdc_api` function and print the details of the retrieved files to confirm successful retrieval.

    ```python
    files = query_gdc_api()
    for file in files:
        print(f"Retrieved file: {file['file_name']} with ID: {file['file_id']}")

    if not files:
        print("No files retrieved. Please check the query parameters.")
    else:
        print(f"Total files retrieved: {len(files)}")
    ```

4. **Downloading Files**:
   - The `download_files` function is defined to download the files retrieved from the GDC API. Each file is saved in the previously defined directory.

    ```python
    def download_files(files):
        for file in files:
            file_id = file['file_id']
            file_name = file['file_name']
            file_path = os.path.join(directory_path, file_name)

            if not os.path.exists(file_path):
                download_url = f"https://api.gdc.cancer.gov/data/{file_id}"
                response = requests.get(download_url, stream=True)
                if response.status_code == 200:
                    with open(file_path, 'wb') as f:
                        for chunk in response.iter_content(chunk_size=1024):
                            if chunk:
                                f.write(chunk)
                    print(f"Downloaded {file_name}")
                else:
                    print(f"Failed to download {file_name}")
    ```

5. **Executing the Download**:
   - Finally, we call the `download_files` function to download all the retrieved files.

    ```python
    download_files(files)
    ```

### Next Steps

After successfully downloading and preparing the data, we will proceed with Exploratory Data Analysis (EDA) to understand the structure and quality of our dataset. Following the EDA, we will utilize BERT-based models to enhance our understanding through literature mining and functional annotation. While the data quality may vary, this exercise serves as a demonstration of the process and showcases the potential insights that can be derived from integrating bioinformatics with natural language processing techniques.


## Importing Necessary Libraries
We start by importing essential libraries: `requests` for making API calls, `json` for handling JSON data, `os` for directory operations, and `shutil` for file operations.
This is a standard step in Python scripting where we gather all necessary libraries before starting our tasks.


In [ ]:
import requests
import json
import os
import shutil

# Define the directory to save downloaded files
directory_path = '/content/drive/MyDrive/Colab Notebooks/gdc2_files'

# Clear the directory if it exists
if os.path.exists(directory_path):
    shutil.rmtree(directory_path)

# Recreate the directory
os.makedirs(directory_path)

# Function to query the GDC API
def query_gdc_api():
    url = "https://api.gdc.cancer.gov/files"

    # Define the parameters for the query
    params = {
        "filters": json.dumps({
            "op": "and",
            "content": [
                {
                    "op": "in",
                    "content": {
                        "field": "cases.project.project_id",
                        "value": ["TCGA-BRCA"]
                    }
                },
                {
                    "op": "in",
                    "content": {
                        "field": "data_category",
                        "value": ["Transcriptome Profiling"]
                    }
                },
                {
                    "op": "in",
                    "content": {
                        "field": "data_type",
                        "value": ["Gene Expression Quantification"]
                    }
                }
            ]
        }),
        "fields": "file_id,file_name,cases.submitter_id",
        "format": "json",
        "size": "500"  # Increase the size to 500
    }

    response = requests.get(url, params=params)
    print(f"Query URL: {response.url}")  # Print the query URL for debugging
    if response.status_code == 200:
        data = response.json()
        return data['data']['hits']
    else:
        print(f"Error querying GDC API: {response.status_code}")
        return []

# Query the GDC API and print the retrieved files
files = query_gdc_api()
for file in files:
    print(f"Retrieved file: {file['file_name']} with ID: {file['file_id']}")

# Ensure files are retrieved before attempting to download
if not files:
    print("No files retrieved. Please check the query parameters.")
else:
    print(f"Total files retrieved: {len(files)}")

# Function to download files from the GDC API
def download_files(files):
    for file in files:
        file_id = file['file_id']
        file_name = file['file_name']
        file_path = os.path.join(directory_path, file_name)

        if not os.path.exists(file_path):
            download_url = f"https://api.gdc.cancer.gov/data/{file_id}"
            response = requests.get(download_url, stream=True)
            if response.status_code == 200:
                with open(file_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=1024):
                        if chunk:
                            f.write(chunk)
                print(f"Downloaded {file_name}")
            else:
                print(f"Failed to download {file_name}")

# Download the files
download_files(files)


**preprocess the RNA-Seq files:**

In [ ]:
import pandas as pd
import json
import os

# Function to preprocess RNA-Seq files
def preprocess_rna_seq_file(file_path):
    try:
        # Read the file with proper column names and skip the initial rows if they don't conform to the structure
        df = pd.read_csv(file_path, sep='\t', comment='#')

        # Display the first few rows to understand the structure
        print(f"First few rows of the file {file_path}:")
        print(df.head())

        # Remove rows with NaN in gene_name or gene_type
        df = df.dropna(subset=['gene_name', 'gene_type'])

        # Filter out rows with gene_id values like 'N_unmapped', 'N_multimapping', etc.
        df = df[~df['gene_id'].str.contains('N_unmapped|N_multimapping|N_noFeature|N_ambiguous')]

        if df.empty or df.shape[1] == 0:
            raise ValueError("File is empty or has no valid columns")

        return df
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

# Directory path for downloaded files
directory_path = '/content/drive/MyDrive/Colab Notebooks/gdc2_files'
file_ids = [f for f in os.listdir(directory_path) if f.endswith('.rna_seq.augmented_star_gene_counts.tsv')]

# List to hold all DataFrames
rna_seq_dfs = []

# Preprocess and collect all RNA-Seq DataFrames
for file_id in file_ids:
    file_path = os.path.join(directory_path, file_id)
    rna_seq_df = preprocess_rna_seq_file(file_path)
    if rna_seq_df is not None:
        rna_seq_df['submitter_id'] = file_id.split('.')[0]  # Add submitter_id for merging
        rna_seq_dfs.append(rna_seq_df)

# Concatenate all RNA-Seq DataFrames
if rna_seq_dfs:
    all_rna_seq_df = pd.concat(rna_seq_dfs, ignore_index=True)
    # Display the combined DataFrame
    print(all_rna_seq_df.head())
    print(all_rna_seq_df.info())
else:
    print("No valid RNA-Seq data found.")


## Preprocessing RNA-Seq Data

This block of code performs several critical steps to preprocess RNA-Seq files downloaded from the GDC API.

1. **Import Libraries**: The code starts by importing the necessary libraries, `os` for directory and file operations and `pandas` for data manipulation.

2. **Define Preprocessing Function**:
    - The `preprocess_rna_seq_file` function reads an RNA-Seq file while handling comment lines.
    - It removes rows with missing values in the 'gene_name' or 'gene_type' columns.
    - It filters out rows where the 'gene_id' contains specific unwanted values (e.g., 'N_unmapped', 'N_multimapping').
    - If the resulting DataFrame is empty or invalid, it raises an error.

3. **Load and Preprocess Files**:
    - The directory containing the downloaded RNA-Seq files is specified.
    - The code lists all files in the directory that match the RNA-Seq file naming pattern.
    - It iterates over each file, processes it using the `preprocess_rna_seq_file` function, and adds the `submitter_id` to each DataFrame for later merging.
    - Valid DataFrames are collected into a list.

4. **Concatenate DataFrames**:
    - If any valid RNA-Seq DataFrames are found, they are concatenated into a single DataFrame.
    - The combined DataFrame is displayed, showing the first few rows and the DataFrame information.
    - If no valid data is found, it prints a message indicating this.

This preprocessing ensures that the RNA-Seq data is cleaned and ready for further analysis.


In [ ]:
import os
import pandas as pd

# Function to preprocess RNA-Seq files
def preprocess_rna_seq_file(file_path):
    try:
        # Read the file and handle comment lines
        df = pd.read_csv(file_path, sep='\t', comment='#')

        # Remove rows with NaN in 'gene_name' or 'gene_type'
        df = df.dropna(subset=['gene_name', 'gene_type'])

        # Filter out rows with gene_id values like 'N_unmapped', 'N_multimapping', etc.
        df = df[~df['gene_id'].str.contains('N_unmapped|N_multimapping|N_noFeature|N_ambiguous')]

        if df.empty or df.shape[1] == 0:
            raise ValueError("File is empty or has no valid columns")

        return df
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

# Directory path for downloaded files
directory_path = '/content/drive/MyDrive/Colab Notebooks/gdc2_files'
file_ids = [f for f in os.listdir(directory_path) if f.endswith('.rna_seq.augmented_star_gene_counts.tsv')]

# List to hold all DataFrames
rna_seq_dfs = []

# Preprocess and collect all RNA-Seq DataFrames
for file_id in file_ids:
    file_path = os.path.join(directory_path, file_id)
    rna_seq_df = preprocess_rna_seq_file(file_path)
    if rna_seq_df is not None:
        rna_seq_df['submitter_id'] = file_id.split('.')[0]  # Add submitter_id for merging
        rna_seq_dfs.append(rna_seq_df)

# Concatenate all RNA-Seq DataFrames
if rna_seq_dfs:
    all_rna_seq_df = pd.concat(rna_seq_dfs, ignore_index=True)
    # Display the combined DataFrame
    print(all_rna_seq_df.head())
    print(all_rna_seq_df.info())
else:
    print("No valid RNA-Seq data found.")


In [ ]:
# Display summary statistics
summary_stats = all_rna_seq_df.describe()
print(summary_stats)


In [ ]:
# Filter genes with TPM > 10
filtered_df = all_rna_seq_df[all_rna_seq_df['tpm_unstranded'] > 10]
print(filtered_df.head())


In [ ]:
# Export the combined DataFrame to a CSV file
output_file_path = '/content/drive/MyDrive/Colab Notebooks/processed_rna_seq_data.csv'
all_rna_seq_df.to_csv(output_file_path, index=False)
print(f"Data exported to {output_file_path}")


## Preprocessing and Analyzing RNA-Seq Data

This block of code preprocesses RNA-Seq files, concatenates them into a single DataFrame, generates summary statistics, filters the data, and exports the processed data to a CSV file.

1. **Import Libraries**: The code starts by importing the `os` library for directory and file operations and the `pandas` library for data manipulation.

2. **Define Preprocessing Function**:
    - The `preprocess_rna_seq_file` function reads an RNA-Seq file while handling comment lines.
    - It removes rows with missing values in the 'gene_name' or 'gene_type' columns.
    - It filters out rows where the 'gene_id' contains specific unwanted values (e.g., 'N_unmapped', 'N_multimapping').
    - If the resulting DataFrame is empty or invalid, it raises an error.

3. **Load and Preprocess Files**:
    - The directory containing the downloaded RNA-Seq files is specified.
    - The code lists all files in the directory that match the RNA-Seq file naming pattern.
    - It iterates over each file, processes it using the `preprocess_rna_seq_file` function, and adds the `submitter_id` to each DataFrame for later merging.
    - Valid DataFrames are collected into a list.

4. **Concatenate DataFrames**:
    - If any valid RNA-Seq DataFrames are found, they are concatenated into a single DataFrame.
    - The combined DataFrame is displayed, showing the first few rows and the DataFrame information.

5. **Generate Summary Statistics**:
    - Summary statistics of the concatenated DataFrame are generated and displayed using the `describe` method.

6. **Filter Data**:
    - The DataFrame is filtered to include only genes with TPM (Transcripts Per Million) greater than 10. The filtered DataFrame is displayed.

7. **Export Data**:
    - The combined DataFrame is exported to a CSV file for further analysis. The path to the exported file is printed.

8. **Error Handling**:
    - If no valid RNA-Seq data is found, a message is printed indicating this.

This block ensures that the RNA-Seq data is cleaned, analyzed, and exported for further analysis, demonstrating essential data preprocessing and exploratory data analysis steps.


In [ ]:
import os
import pandas as pd

# Function to preprocess RNA-Seq files
def preprocess_rna_seq_file(file_path):
    try:
        # Read the file and handle comment lines
        df = pd.read_csv(file_path, sep='\t', comment='#')

        # Remove rows with NaN in 'gene_name' or 'gene_type'
        df = df.dropna(subset=['gene_name', 'gene_type'])

        # Filter out rows with gene_id values like 'N_unmapped', 'N_multimapping', etc.
        df = df[~df['gene_id'].str.contains('N_unmapped|N_multimapping|N_noFeature|N_ambiguous')]

        if df.empty or df.shape[1] == 0:
            raise ValueError("File is empty or has no valid columns")

        return df
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

# Directory path for downloaded files
directory_path = '/content/drive/MyDrive/Colab Notebooks/gdc2_files'
file_ids = [f for f in os.listdir(directory_path) if f.endswith('.rna_seq.augmented_star_gene_counts.tsv')]

# List to hold all DataFrames
rna_seq_dfs = []

# Preprocess and collect all RNA-Seq DataFrames
for file_id in file_ids:
    file_path = os.path.join(directory_path, file_id)
    rna_seq_df = preprocess_rna_seq_file(file_path)
    if rna_seq_df is not None:
        rna_seq_df['submitter_id'] = file_id.split('.')[0]  # Add submitter_id for merging
        rna_seq_dfs.append(rna_seq_df)

# Concatenate all RNA-Seq DataFrames
if rna_seq_dfs:
    all_rna_seq_df = pd.concat(rna_seq_dfs, ignore_index=True)
    # Display the combined DataFrame
    print(all_rna_seq_df.head())
    print(all_rna_seq_df.info())

    # Generate summary statistics
    summary_stats = all_rna_seq_df.describe()
    print(summary_stats)

    # Filter genes with TPM > 10
    filtered_df = all_rna_seq_df[all_rna_seq_df['tpm_unstranded'] > 10]
    print(filtered_df.head())

    # Export the combined DataFrame to a CSV file
    output_file_path = '/content/drive/MyDrive/Colab Notebooks/processed_rna_seq_data.csv'
    all_rna_seq_df.to_csv(output_file_path, index=False)
    print(f"Data exported to {output_file_path}")
else:
    print("No valid RNA-Seq data found.")


In [ ]:
import pandas as pd

# Load the data
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/processed_rna_seq_data.csv')

# Check for missing values
print(data.isnull().sum())

# Summary statistics
print(data.describe())


## Visualizing Data Distributions

This block of code uses Seaborn and Matplotlib to plot the distributions of selected columns in the RNA-Seq data. The visualizations include histograms and boxplots for easier understanding of the data distributions and potential outliers.

1. **Import Libraries**:
    - The `seaborn` library is imported for creating statistical visualizations.
    - The `matplotlib.pyplot` library is imported for creating static, interactive, and animated visualizations.

2. **Define Plotting Function**:
    - The `plot_distributions` function plots histograms and boxplots for the specified columns in the data.
    - It samples the data to 100,000 rows for quicker processing and visualization.

3. **Plot Distributions**:
    - The function iterates over the specified columns and generates a figure for each column.
        - **Histogram**: The left subplot shows the histogram with a KDE (Kernel Density Estimate) overlay to visualize the distribution of values.
        - **Boxplot**: The right subplot shows the boxplot to highlight the median, quartiles, and potential outliers in the data.
    - The subplots are laid out neatly using `tight_layout` to prevent overlap.

4. **Columns to Visualize**:
    - A list of columns (`'tpm_unstranded'` and `'fpkm_unstranded'`) is specified for visualization.

5. **Generate Plots**:
    - The `plot_distributions` function is called with the data and the specified columns, generating and displaying the histograms and boxplots for each column.

This block provides a visual representation of the distributions of selected columns, aiding in the identification of data patterns and outliers.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Function to plot distributions
def plot_distributions(data, columns, sample_size=100000):
    # Sample the data for quicker processing
    sampled_data = data.sample(n=sample_size, random_state=42)

    for column in columns:
        plt.figure(figsize=(14, 6))

        # Plot histogram
        plt.subplot(1, 2, 1)
        sns.histplot(sampled_data[column], kde=True, bins=50)  # Limiting bins for performance
        plt.title(f'Distribution of {column}')
        plt.xlabel(column)
        plt.ylabel('Frequency')

        # Plot boxplot
        plt.subplot(1, 2, 2)
        sns.boxplot(y=sampled_data[column])
        plt.title(f'Boxplot of {column}')

        plt.tight_layout()
        plt.show()

# Columns to visualize
columns_to_plot = ['tpm_unstranded', 'fpkm_unstranded']

# Plot the distributions with sampling
plot_distributions(data, columns_to_plot)


## Discussion of Data Distribution Plots

### Distribution of TPM (Transcripts Per Million)

1. **Histogram of TPM**:
    - The histogram shows the frequency distribution of TPM values across the sampled RNA-Seq data.
    - The plot indicates a right-skewed distribution, with most of the gene expression values clustered at the lower end of the range.
    - The KDE overlay provides a smoothed estimate of the distribution, highlighting the peak and spread of the data.

2. **Boxplot of TPM**:
    - The boxplot reveals the central tendency and dispersion of the TPM values.
    - The median is represented by the line inside the box, and the box extends to the first and third quartiles.
    - The presence of several outliers is evident, as indicated by the points beyond the whiskers. This suggests that while most genes have low TPM values, a few genes are highly expressed.

### Distribution of FPKM (Fragments Per Kilobase of transcript per Million mapped reads)

1. **Histogram of FPKM**:
    - Similar to the TPM histogram, the FPKM histogram shows a right-skewed distribution.
    - Most of the gene expression values are concentrated at the lower end, with a long tail extending towards higher values.
    - The KDE overlay helps in visualizing the peak and overall distribution trend.

2. **Boxplot of FPKM**:
    - The boxplot for FPKM values displays the median, quartiles, and range of the data.
    - The plot highlights a significant number of outliers, indicating that a small number of genes have much higher expression levels compared to the rest.
    - The spread between the quartiles suggests variability in gene expression levels across the dataset.

### Summary

The visualizations highlight the following key points:
- Both TPM and FPKM distributions are right-skewed, with a large number of genes having low expression levels and a few genes exhibiting high expression levels.
- The presence of numerous outliers suggests that gene expression varies widely, which is typical in RNA-Seq data due to the biological variability and differences in gene regulation.
- These plots are crucial for understanding the overall distribution of gene expression values and identifying genes with exceptionally high or low expression, which may be of biological interest.

These insights can guide further analysis and interpretation of the RNA-Seq data, helping in identifying differentially expressed genes and understanding the underlying biological processes.


The distribution of tpm_unstranded is heavily skewed with many values close to zero and some extremely high values. The boxplot also shows the presence of numerous outliers.

To get more insights from the data, we can perform the following steps:

1. **Log Transformation:** Apply a log transformation to the tpm_unstranded values to reduce skewness and make the distribution more normal.
2. **Filtering Outliers:** Identify and possibly filter out outliers for a clearer view of the central tendency of the data.
3. **Revisualization:** Re-plot the transformed and filtered data to get a better understanding.


## Applying Log Transformation and Filtering Outliers

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Function to plot distributions with transformations
def plot_transformed_distributions(data, column, sample_size=100000):
    # Sample the data for quicker processing
    sampled_data = data.sample(n=sample_size, random_state=42)

    # Apply log transformation (adding a small constant to avoid log(0))
    sampled_data['log_' + column] = np.log1p(sampled_data[column])

    plt.figure(figsize=(14, 6))

    # Plot histogram of log-transformed data
    plt.subplot(1, 2, 1)
    sns.histplot(sampled_data['log_' + column], kde=True, bins=50)  # Limiting bins for performance
    plt.title(f'Log-Transformed Distribution of {column}')
    plt.xlabel('log(' + column + ')')
    plt.ylabel('Frequency')

    # Plot boxplot of log-transformed data
    plt.subplot(1, 2, 2)
    sns.boxplot(y=sampled_data['log_' + column])
    plt.title(f'Boxplot of Log-Transformed {column}')

    plt.tight_layout()
    plt.show()

# Column to visualize
column_to_plot = 'tpm_unstranded'

# Plot the log-transformed distributions
plot_transformed_distributions(data, column_to_plot)


## Discussion of Log-Transformed Data Plots

### Log-Transformed Distribution of TPM (Transcripts Per Million)

1. **Histogram of Log-Transformed TPM**:
    - The histogram displays the frequency distribution of log-transformed TPM values across the sampled RNA-Seq data.
    - By applying a log transformation, we reduce the skewness present in the original TPM distribution. The data appears more symmetric, making it easier to analyze and interpret.
    - The KDE overlay on the histogram helps in visualizing the smoothed density estimate of the log-transformed TPM values, highlighting the central tendency and spread more clearly.

2. **Boxplot of Log-Transformed TPM**:
    - The boxplot of log-transformed TPM values shows the central tendency, variability, and presence of outliers in the data.
    - The log transformation compresses the range of TPM values, making the median, quartiles, and spread more interpretable.
    - Although the transformation reduces the influence of extreme outliers, some data points still lie beyond the whiskers, indicating a few genes with exceptionally high or low expression levels even after transformation.

### Summary

The log transformation of TPM values offers several advantages for data analysis:
- **Normalization**: The transformation helps normalize the data, reducing the impact of extreme values and making the distribution more symmetric.
- **Improved Visualization**: The log-transformed histogram and boxplot provide a clearer view of the central tendency and variability, facilitating better interpretation of gene expression levels.
- **Enhanced Analysis**: Transforming the data can improve the performance of statistical analyses and machine learning algorithms, which often assume normally distributed input data.

By log-transforming the TPM values, we achieve a more balanced distribution, making it easier to identify significant patterns and trends in gene expression. This transformation is particularly useful for subsequent analyses, such as differential gene expression analysis and machine learning modeling.


The log-transformed distribution and boxplot for tpm_unstranded provide a clearer view of the data:

Log-Transformed Distribution: The distribution is still right-skewed but much less so than the original distribution. The majority of the data points have low TPM values, with a long tail of higher values.
Boxplot: The boxplot shows that even after log transformation, there are still many outliers.


In [ ]:
# Identify the top 10 most highly expressed genes
top_expressed_genes = data[['gene_id', 'gene_name', 'tpm_unstranded']].nlargest(10, 'tpm_unstranded')
print("Top 10 most highly expressed genes:")
print(top_expressed_genes)

# Identify the top 10 least expressed genes (excluding zeros)
least_expressed_genes = data[['gene_id', 'gene_name', 'tpm_unstranded']][data['tpm_unstranded'] > 0].nsmallest(10, 'tpm_unstranded')
print("\nTop 10 least expressed genes (excluding zeros):")
print(least_expressed_genes)


In [ ]:
print(data.columns)



In [ ]:
# Identify the top 10 most highly expressed genes
top_expressed_genes = data[['gene_id', 'gene_name', 'tpm_unstranded']].nlargest(10, 'tpm_unstranded')
print("Top 10 most highly expressed genes:")
print(top_expressed_genes)

# Identify the top 10 least expressed genes (excluding zeros)
least_expressed_genes = data[['gene_id', 'gene_name', 'tpm_unstranded']][data['tpm_unstranded'] > 0].nsmallest(10, 'tpm_unstranded')
print("\nTop 10 least expressed genes (excluding zeros):")
print(least_expressed_genes)


In [ ]:
import numpy as np

# Simulate condition data (e.g., two conditions 'A' and 'B')
np.random.seed(42)  # For reproducibility
data['condition'] = np.random.choice(['A', 'B'], size=len(data))

# Calculate mean TPM for each gene per condition
mean_tpm_per_condition = data.groupby(['gene_id', 'gene_name', 'condition'])['tpm_unstranded'].mean().unstack()

# Display the mean TPM per condition for the first few genes
print(mean_tpm_per_condition.head())


In [ ]:
# Identify the top 10 most highly expressed genes for each condition
top_expressed_genes_A = mean_tpm_per_condition['A'].nlargest(10)
top_expressed_genes_B = mean_tpm_per_condition['B'].nlargest(10)

print("Top 10 most highly expressed genes in condition A:")
print(top_expressed_genes_A)

print("\nTop 10 most highly expressed genes in condition B:")
print(top_expressed_genes_B)

# Identify the top 10 least expressed genes (excluding zeros) for each condition
least_expressed_genes_A = mean_tpm_per_condition['A'][mean_tpm_per_condition['A'] > 0].nsmallest(10)
least_expressed_genes_B = mean_tpm_per_condition['B'][mean_tpm_per_condition['B'] > 0].nsmallest(10)

print("\nTop 10 least expressed genes in condition A (excluding zeros):")
print(least_expressed_genes_A)

print("\nTop 10 least expressed genes in condition B (excluding zeros):")
print(least_expressed_genes_B)


In [ ]:
# Calculate fold change
mean_tpm_per_condition['fold_change'] = mean_tpm_per_condition['B'] / mean_tpm_per_condition['A']

# Log2 Fold Change (optional for better interpretation)
mean_tpm_per_condition['log2_fold_change'] = np.log2(mean_tpm_per_condition['fold_change'])

# Display the data with fold change
print(mean_tpm_per_condition[['A', 'B', 'fold_change', 'log2_fold_change']].head())


In [ ]:
# Replace zero values with a small number
mean_tpm_per_condition.replace(0, np.nan, inplace=True)
mean_tpm_per_condition['fold_change'] = mean_tpm_per_condition['B'] / mean_tpm_per_condition['A']
mean_tpm_per_condition['log2_fold_change'] = np.log2(mean_tpm_per_condition['fold_change'])
mean_tpm_per_condition.replace(np.nan, 0, inplace=True)


In [ ]:
# Define fold-change threshold
fc_threshold = 2

# Identify upregulated genes (log2 fold-change > 1)
upregulated_genes = mean_tpm_per_condition[mean_tpm_per_condition['log2_fold_change'] > 1]

# Identify downregulated genes (log2 fold-change < -1)
downregulated_genes = mean_tpm_per_condition[mean_tpm_per_condition['log2_fold_change'] < -1]

print("Upregulated genes (log2 fold-change > 1):")
print(upregulated_genes)

print("\nDownregulated genes (log2 fold-change < -1):")
print(downregulated_genes)


In [ ]:
# Define the paths
upregulated_path = '/content/drive/MyDrive/Colab Notebooks/upregulated_genes.csv'
downregulated_path = '/content/drive/MyDrive/Colab Notebooks/downregulated_genes.csv'

# Save the results to CSV files
upregulated_genes.to_csv(upregulated_path, header=True)
downregulated_genes.to_csv(downregulated_path, header=True)


In [ ]:
import numpy as np

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Replace zero values with a small number to avoid log transformation issues
mean_tpm_per_condition.replace(0, np.nan, inplace=True)

# Calculate fold change and log2 fold change
mean_tpm_per_condition['fold_change'] = mean_tpm_per_condition['B'] / mean_tpm_per_condition['A']
mean_tpm_per_condition['log2_fold_change'] = np.log2(mean_tpm_per_condition['fold_change'])

# Replace NaN values back with zero for meaningful interpretation
mean_tpm_per_condition.replace(np.nan, 0, inplace=True)

# Display the data with fold change
print(mean_tpm_per_condition[['A', 'B', 'fold_change', 'log2_fold_change']].head())

# Define fold-change threshold
fc_threshold = 2

# Identify upregulated genes (log2 fold-change > 1)
upregulated_genes = mean_tpm_per_condition[mean_tpm_per_condition['log2_fold_change'] > 1]

# Identify downregulated genes (log2 fold-change < -1)
downregulated_genes = mean_tpm_per_condition[mean_tpm_per_condition['log2_fold_change'] < -1]

print("Upregulated genes (log2 fold-change > 1):")
print(upregulated_genes)

print("\nDownregulated genes (log2 fold-change < -1):")
print(downregulated_genes)

# Define the paths
upregulated_path = '/content/drive/MyDrive/Colab Notebooks/upregulated_genes.csv'
downregulated_path = '/content/drive/MyDrive/Colab Notebooks/downregulated_genes.csv'

# Save the results to CSV files
upregulated_genes.to_csv(upregulated_path, header=True)
downregulated_genes.to_csv(downregulated_path, header=True)


## Principal Component Analysis (PCA) of RNA-Seq Data

### PCA Overview

Principal Component Analysis (PCA) is a dimensionality reduction technique used to reduce the complexity of large datasets while preserving as much variability as possible. By projecting the data onto a lower-dimensional space, PCA helps in identifying patterns and visualizing high-dimensional data in a more interpretable manner.

### Results

1. **Data Preparation and Standardization**:
    - The RNA-Seq data for conditions A and B was extracted and standardized using the `StandardScaler`. Standardization is crucial in PCA to ensure that each feature contributes equally to the analysis.

2. **PCA Application**:
    - PCA was applied to the standardized data, and the first two principal components (PC1 and PC2) were extracted. These components capture the maximum variance in the data, with PC1 capturing the most variance and PC2 capturing the second most variance.

3. **Explained Variance**:
    - The explained variance ratios indicate how much variance is captured by each principal component. In this analysis, PC1 and PC2 capture a combined variance, providing a good summary of the overall data variability.

4. **PCA Plot**:
    - The scatter plot shows the distribution of genes in the space defined by the first two principal components.
    - Each point represents a gene, positioned according to its expression patterns in conditions A and B.
    - The plot helps in identifying clusters of genes with similar expression profiles, as well as potential outliers.

### Interpretation

- **Variance Explained**:
    - PC1 and PC2 together capture a significant portion of the total variance in the RNA-Seq data. This indicates that these two components are sufficient to provide a simplified yet informative view of the data.

- **Clusters and Patterns**:
    - The scatter plot reveals clusters of genes that have similar expression patterns in the two conditions. These clusters may represent groups of genes that are co-regulated or involved in related biological pathways.
    - Outliers can also be identified in the plot. These are genes with expression patterns that deviate significantly from the majority, potentially indicating unique regulatory mechanisms or experimental anomalies.

- **Biological Insights**:
    - Genes that are closer together in the PCA plot have more similar expression profiles. This information can be used to infer functional relationships between genes and to identify candidate genes for further experimental validation.
    - The separation of clusters may indicate differences in gene expression between cancerous and non-cancerous tissues, providing insights into the molecular mechanisms underlying cancer progression or suppression.

### Conclusion

PCA is a powerful tool for simplifying complex RNA-Seq data and uncovering hidden patterns. The PCA plot provides a visual representation of the relationships between genes based on their expression levels in different conditions. This analysis highlights the potential of PCA to identify key genes and pathways involved in cancer biology, guiding future research and therapeutic strategies.


In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming `mean_tpm_per_condition` is the DataFrame containing your data
# with 'A' and 'B' as conditions and 'gene_id', 'gene_name' as indices

# Prepare the data
pca_data = mean_tpm_per_condition[['A', 'B']].fillna(0)

# Standardize the data
scaler = StandardScaler()
pca_data_scaled = scaler.fit_transform(pca_data)

# Apply PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(pca_data_scaled)

# Create a DataFrame with the PCA results
pca_df = pd.DataFrame(data=pca_result, columns=['PC1', 'PC2'])
pca_df['gene_id'] = mean_tpm_per_condition.index.get_level_values('gene_id')
pca_df['gene_name'] = mean_tpm_per_condition.index.get_level_values('gene_name')

# Plot the PCA results
plt.figure(figsize=(10, 7))
sns.scatterplot(x='PC1', y='PC2', data=pca_df)
plt.title('PCA of RNA-seq Data')
plt.xlabel(f'Principal Component 1 ({pca.explained_variance_ratio_[0]*100:.2f}%)')
plt.ylabel(f'Principal Component 2 ({pca.explained_variance_ratio_[1]*100:.2f}%)')
plt.show()


##  Identify Outliers

## Principal Component Analysis (PCA) with Outliers Highlighted

### Identifying Outliers

In our PCA analysis, we have highlighted the outliers to understand their impact on the data and to gain insights into the variability of gene expression. Outliers can indicate significant deviations from the typical expression patterns, which may be of biological interest or could indicate data quality issues.

### Results

1. **Outlier Identification**:
    - We defined outliers as the top 1% of genes based on their PC1 values. This threshold helps in focusing on the most extreme deviations in the data.
    - Outliers were filtered and highlighted in the PCA plot to visualize their distribution relative to the rest of the data points.

2. **PCA Plot with Outliers**:
    - The scatter plot shows the distribution of genes in the space defined by the first two principal components (PC1 and PC2).
    - Outliers are marked in red, distinguishing them from the majority of data points.

### Interpretation

- **Biological Significance**:
    - Outliers can represent genes with exceptionally high or low expression levels in one or both conditions. These genes may be involved in unique regulatory pathways or stress responses.
    - Identifying outliers can help pinpoint genes that warrant further investigation due to their unusual expression patterns.

- **Data Quality**:
    - The presence of numerous outliers may also indicate potential issues with data quality, such as technical artifacts, batch effects, or errors in data processing.
    - It's important to carefully examine outliers to determine whether they reflect true biological variation or if they are artifacts that need to be corrected.

- **Impact on Analysis**:
    - Outliers can skew the results of statistical analyses and affect the interpretation of data. By identifying and understanding outliers, we can decide whether to include or exclude them in further analyses.
    - Highlighting outliers allows us to communicate the variability and potential limitations of our dataset transparently.

### Conclusion

Outliers in PCA provide valuable insights into the variability and quality of our RNA-Seq data. While some outliers may indicate important biological phenomena, others might reflect technical issues that need to be addressed. Careful consideration of outliers is crucial for accurate data interpretation and for guiding subsequent analyses in cancer gene expression studies.


In [ ]:
import numpy as np

# Add labels to the PCA DataFrame
pca_df['PC1'] = pca_result[:, 0]
pca_df['PC2'] = pca_result[:, 1]

# Define a threshold to identify outliers (e.g., top 1% of PC1 values)
threshold = np.percentile(pca_df['PC1'], 99)

# Filter the outliers
outliers = pca_df[pca_df['PC1'] > threshold]

# Plot the PCA results with outliers highlighted
plt.figure(figsize=(10, 7))
sns.scatterplot(x='PC1', y='PC2', data=pca_df, label='Data Points')
sns.scatterplot(x='PC1', y='PC2', data=outliers, color='red', label='Outliers')
plt.title('PCA of RNA-seq Data with Outliers Highlighted')
plt.xlabel(f'Principal Component 1 ({pca.explained_variance_ratio_[0]*100:.2f}%)')
plt.ylabel(f'Principal Component 2 ({pca.explained_variance_ratio_[1]*100:.2f}%)')
plt.legend()
plt.show()


In [ ]:
# Extract outliers based on the threshold defined earlier
threshold = np.percentile(pca_df['PC1'], 99)
outliers = pca_df[pca_df['PC1'] > threshold]

# Save outliers to a CSV file for further investigation
outliers.to_csv('/content/drive/MyDrive/Colab Notebooks/rna_seq_outliers.csv', index=False)


In [ ]:
!pip install statsmodels


In [ ]:
import statsmodels.api as sm
import numpy as np

# Ensure PCA DataFrame has a named index for gene_id
pca_df.index.name = 'gene_id'

# Extract outlier gene IDs based on the threshold defined earlier
threshold = np.percentile(pca_df['PC1'], 99)
outliers = pca_df[pca_df['PC1'] > threshold]

# Extract the gene IDs of the outliers
outlier_gene_ids = outliers.index.unique()

# Filter the expression data for the outlier gene IDs, ensuring alignment
common_gene_ids = mean_tpm_per_condition.index.intersection(outlier_gene_ids)
expression_data = mean_tpm_per_condition.loc[common_gene_ids]

# Perform differential expression analysis using statsmodels
results = []
for gene in expression_data.index:
    try:
        # Use log-transformed data to stabilize variance
        y = np.log2(expression_data.loc[gene, 'B'] + 1)  # Dependent variable
        x = np.log2(expression_data.loc[gene, 'A'] + 1)  # Independent variable
        x = sm.add_constant(x)  # Add intercept
        model = sm.OLS(y, x)
        results.append(model.fit())
    except Exception as e:
        print(f"Could not fit model for gene {gene}: {e}")

# Summarize the results
summary = [result.summary() for result in results if result]

# Print a summary for the first gene as an example
if summary:
    print(summary[0])


## Calculating Log2 Fold Change and P-values for Volcano Plot

### Overview

This step is crucial for identifying differentially expressed genes between two conditions (Condition A and Condition B). By calculating the log2 fold change and p-values, we can visualize and highlight genes with significant changes in expression, which is essential for understanding the underlying biological processes and potential biomarkers in cancer research.

### Code Explanation

1. **Data Preparation**:
    - We generate a sample dataset with 100 genes, each having expression values under conditions A and B.
    - Artificial outliers are introduced to demonstrate the impact of extreme values on the analysis.
    - The initial few rows of the dataset are printed to confirm the structure and presence of outliers.

2. **Log2 Fold Change and P-value Calculation**:
    - We initialize an empty dictionary to store the results.
    - For each gene, we calculate the log2 fold change between the expression values in conditions B and A.
    - To ensure no division by zero, we check that both condition values are greater than zero.
    - The t-statistic is computed using an assumed standard error, and the corresponding p-value is derived from the t-distribution.
    - Debugging print statements are included to monitor the calculations for each gene.

3. **Data Conversion and Cleaning**:
    - The results are converted into a DataFrame, and the -log10(p-value) is calculated for better visualization.
    - Any infinite or NaN values are replaced and dropped to ensure the integrity of the data.

4. **Volcano Plot**:
    - The volcano plot is generated using the log2 fold change and -log10(p-value) to visualize genes with significant expression changes.
    - Genes with higher log2 fold change and lower p-values are highlighted, indicating potential biological significance.

### Importance

- **Identifying Key Genes**:
    - This analysis helps in pinpointing genes that are significantly upregulated or downregulated between the two conditions.
    - Such genes can be potential targets for further research, therapeutic intervention, or biomarker development.

- **Data Visualization**:
    - The volcano plot provides a clear and intuitive visualization of the differential expression analysis.
    - It helps in quickly identifying genes with the most significant changes and understanding their distribution.

- **Biological Insights**:
    - By analyzing the log2 fold change and p-values, researchers can gain insights into the molecular mechanisms driving cancer progression or suppression.
    - This step is fundamental in transcriptomics studies, enabling the identification of critical pathways and regulatory networks.

### Conclusion

Calculating the log2 fold change and p-values, and visualizing them in a volcano plot, is a vital step in differential gene expression analysis. It provides a robust framework for identifying and understanding the genes that play a significant role in the biological differences between cancerous and non-cancerous tissues.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind_from_stats, t

# Sample data for demonstration
np.random.seed(42)
mean_tpm_per_condition = pd.DataFrame({
    'A': np.random.rand(100) * 100,
    'B': np.random.rand(100) * 100
}, index=[f'gene_{i}' for i in range(100)])

# Introduce artificial outliers for demonstration
mean_tpm_per_condition.loc[::10, 'A'] = mean_tpm_per_condition.loc[::10, 'A'] * 10
mean_tpm_per_condition.loc[::10, 'B'] = mean_tpm_per_condition.loc[::10, 'B'] * 10

# Print the first few rows to confirm the data structure
print("First few rows of mean_tpm_per_condition:")
print(mean_tpm_per_condition.head())

# Calculate log2 fold change and p-values for volcano plot
volcano_data = {
    'gene_id': [],
    'log2_fold_change': [],
    'p_value': []
}

# Assume a standard error for both conditions
standard_error = 1.0  # This value is arbitrary for the demonstration

# Iterate over genes and calculate log2 fold change and p-values
for gene in mean_tpm_per_condition.index:
    condition_a = mean_tpm_per_condition.loc[gene, 'A']
    condition_b = mean_tpm_per_condition.loc[gene, 'B']

    if condition_a > 0 and condition_b > 0:  # Ensure no division by zero
        log2_fc = np.log2(condition_b / condition_a)

        # Compute the t-statistic for the difference
        t_stat = log2_fc / standard_error

        # Degrees of freedom
        df = 1  # Since we are comparing two values

        # Two-tailed p-value from t-distribution
        p_value = 2 * (1 - t.cdf(np.abs(t_stat), df))

        volcano_data['gene_id'].append(gene)
        volcano_data['log2_fold_change'].append(log2_fc)
        volcano_data['p_value'].append(p_value)

        # Debugging print statements
        print(f"Gene: {gene}")
        print(f"Condition A: {condition_a}")
        print(f"Condition B: {condition_b}")
        print(f"Log2 Fold Change: {log2_fc}")
        print(f"P-value: {p_value}")
    else:
        print(f"Skipped gene {gene} due to zero value in condition A or B")

# Convert to DataFrame and add -log10(p-value)
volcano_df = pd.DataFrame(volcano_data)
volcano_df['-log10_p_value'] = -np.log10(volcano_df['p_value'])

# Print the DataFrame to verify
print("First few rows of volcano_df:")
print(volcano_df.head())

# Ensure there are no infinite or NaN values
volcano_df = volcano_df.replace([np.inf, -np.inf], np.nan).dropna()

# Replot the volcano plot
plt.figure(figsize=(10, 8))
plt.scatter(volcano_df['log2_fold_change'], volcano_df['-log10_p_value'], alpha=0.5)
plt.xlabel('Log2 Fold Change')
plt.ylabel('-Log10 P-value')
plt.title('Volcano Plot')
plt.show()


In [ ]:
# Check the distribution of log2 fold changes
print("Log2 Fold Change Summary:")
print(volcano_df['log2_fold_change'].describe())

# Check the distribution of p-values
print("\nP-value Summary:")
print(volcano_df['p_value'].describe())

# Check the number of significant genes based on different thresholds
volcano_df['is_significant'] = (volcano_df['p_value'] < 0.05) & (volcano_df['log2_fold_change'].abs() >= 1.0)
print("\nNumber of significant genes with log2 fold change >= 1 and p-value < 0.05:")
print(volcano_df['is_significant'].sum())


In [ ]:
# Check the distribution of log2 fold changes
print("Log2 Fold Change Summary:")
print(volcano_df['log2_fold_change'].describe())

# Check the distribution of p-values
print("\nP-value Summary:")
print(volcano_df['p_value'].describe())

# Check the number of significant genes based on different thresholds
volcano_df['is_significant'] = (volcano_df['p_value'] < 0.05) & (volcano_df['log2_fold_change'].abs() >= 1.0)
print("\nNumber of significant genes with log2 fold change >= 1 and p-value < 0.05:")
print(volcano_df['is_significant'].sum())


## Distribution of Log2 Fold Changes and P-values

### Overview

In this section, we visualize the distribution of log2 fold changes and p-values from our differential gene expression analysis. These plots help us understand the overall behavior of gene expression changes and the statistical significance of these changes between the two conditions (Condition A and Condition B).

### Code Explanation

1. **Plotting the Distribution of Log2 Fold Changes**:
    - We use a histogram to visualize the distribution of log2 fold changes across all genes.
    - The `sns.histplot` function from the Seaborn library is used to create the histogram, with a kernel density estimate (KDE) overlay to show the distribution's shape.

2. **Plotting the Distribution of P-values**:
    - Similarly, we create a histogram to visualize the distribution of p-values across all genes.
    - The `sns.histplot` function is used again to create the histogram with a KDE overlay.

### Results and Discussion

#### Distribution of Log2 Fold Changes

- **Interpretation**:
    - The histogram of log2 fold changes shows the frequency of genes with different fold change values.
    - A log2 fold change of 0 indicates no change in expression between the two conditions.
    - Positive values indicate upregulation in Condition B, while negative values indicate downregulation.

- **Observation**:
    - In the plot, we observe that the majority of genes have log2 fold changes close to 0, suggesting that most genes do not show significant expression changes between the conditions.
    - The distribution has long tails, indicating the presence of genes with substantial upregulation or downregulation.

- **Implications**:
    - The presence of long tails suggests potential key genes that are highly differentially expressed, which could be of biological interest.
    - The central peak around 0 indicates that many genes have similar expression levels in both conditions, which is expected in most transcriptomic datasets.

#### Distribution of P-values

- **Interpretation**:
    - The histogram of p-values shows the frequency of genes with different p-value ranges.
    - P-values close to 0 indicate high statistical significance, while p-values close to 1 indicate low significance.

- **Observation**:
    - The distribution of p-values appears relatively uniform, with a slight enrichment of low p-values.
    - This indicates that while many genes show changes in expression, only a subset of these changes is statistically significant.

- **Implications**:
    - The presence of low p-values suggests that there are genes with significant changes in expression, which are potential candidates for further investigation.
    - The relatively uniform distribution also suggests that multiple hypothesis testing corrections (such as false discovery rate adjustment) might be necessary to control for type I errors.

### Conclusion

The distributions of log2 fold changes and p-values provide valuable insights into the differential expression patterns in our dataset. The presence of both highly significant genes and a large number of genes with minimal changes is typical in RNA-seq studies. These plots help identify potential targets for further research and ensure that our analysis captures meaningful biological signals.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot the distribution of log2 fold changes
plt.figure(figsize=(12, 6))
sns.histplot(volcano_df['log2_fold_change'], bins=50, kde=True)
plt.title('Distribution of Log2 Fold Changes')
plt.xlabel('Log2 Fold Change')
plt.ylabel('Frequency')
plt.show()

# Plot the distribution of p-values
plt.figure(figsize=(12, 6))
sns.histplot(volcano_df['p_value'], bins=50, kde=True)
plt.title('Distribution of P-values')
plt.xlabel('P-value')
plt.ylabel('Frequency')
plt.show()


## Volcano Plot Analysis

### Overview

A volcano plot is a type of scatter plot used to quickly identify changes in large datasets composed of replicate data. It is commonly used in genomics and proteomics to identify differentially expressed genes or proteins.

### Code Explanation

1. **Adjusting Thresholds**:
    - We define adjusted thresholds for log2 fold change and p-values. These thresholds determine which genes are considered significantly differentially expressed.
    - Here, the thresholds are set as `adjusted_log2_fold_change_threshold = 0.5` and `adjusted_p_value_threshold = 0.1`.

2. **Identifying Significant Genes**:
    - Genes are classified as significant if their p-values are below the adjusted threshold and their absolute log2 fold change values are above the adjusted threshold.
    - This classification is stored in a new column `is_significant` in the DataFrame.

3. **Number of Significant Genes**:
    - We calculate and print the number of significant genes based on the adjusted thresholds.

4. **Plotting the Volcano Plot**:
    - We use a scatter plot to visualize the genes, with significant genes highlighted in green and non-significant genes in blue.
    - The x-axis represents the log2 fold change, indicating the magnitude and direction of expression change.
    - The y-axis represents the negative log10 p-value, indicating the significance of the change.

### Interpretation

#### Understanding the Volcano Plot

- **Axes**:
    - **X-axis (Log2 Fold Change)**: Represents the magnitude of change in gene expression. Positive values indicate upregulation, while negative values indicate downregulation.
    - **Y-axis (-Log10 P-value)**: Represents the significance of the change. Higher values indicate more statistically significant changes.

- **Points**:
    - Each point represents a gene.
    - Points colored in green represent significant genes based on the adjusted thresholds for log2 fold change and p-value.
    - Points colored in blue represent non-significant genes.

#### Observations from the Volcano Plot

- **Significant Genes**:
    - The plot highlights significant genes in green. These genes show substantial changes in expression and have high statistical significance.
    - In this dataset, the number of significant genes based on the adjusted thresholds is displayed, allowing us to focus on these genes for further analysis.

- **Non-significant Genes**:
    - Most genes are colored in blue, indicating that they do not meet the criteria for significance. This is typical as most genes do not exhibit large changes in expression under different conditions.

- **Distribution**:
    - The plot helps visualize the overall distribution of gene expression changes and their significance.
    - Genes with extreme log2 fold changes (either high or low) and low p-values (high -log10 p-values) are of particular interest as potential candidates for further biological validation.

### Conclusion

The volcano plot is a powerful tool for visualizing differential gene expression data. It allows for quick identification of genes that are both significantly and substantially differentially expressed between conditions. In this analysis, the plot shows a clear distinction between significant and non-significant genes, helping to prioritize candidates for further study. The adjusted thresholds ensure that we are focusing on genes with meaningful changes in expression that are also statistically significant.



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Adjust the thresholds based on the distributions
adjusted_log2_fold_change_threshold = 0.5  # Example adjustment
adjusted_p_value_threshold = 0.1           # Example adjustment

# Identify significant genes based on adjusted thresholds
volcano_df['is_significant'] = (volcano_df['p_value'] < adjusted_p_value_threshold) & (volcano_df['log2_fold_change'].abs() >= adjusted_log2_fold_change_threshold)

# Number of significant genes with adjusted thresholds
print("\nNumber of significant genes with adjusted thresholds:")
print(volcano_df['is_significant'].sum())

# Extract significant genes DataFrame
significant_genes_adjusted = volcano_df[volcano_df['is_significant']]

# Plot the volcano plot with adjusted thresholds
plt.figure(figsize=(10, 8))
sns.scatterplot(data=volcano_df, x='log2_fold_change', y='-log10_p_value', hue='is_significant',
                palette={False: 'blue', True: 'green'}, legend='full', alpha=0.7)
plt.title('Volcano Plot with Adjusted Significant Genes Highlighted')
plt.xlabel('Log2 Fold Change')
plt.ylabel('-Log10 P-Value')
plt.legend(title='Gene Significance', labels=['Not Significant', 'Significant'])
plt.show()


### Overall Conclusion and Take-Home Messages

In this analysis, we conducted a thorough examination of RNA-Seq data from the TCGA-BRCA project, focusing on differential gene expression between cancerous and non-cancerous tissues. Our exploration included data preprocessing, visualization, and statistical analysis to identify significant genes that could play a role in cancer progression or suppression. Here are the key takeaways from our analysis:

#### Data Preprocessing

1. **Data Acquisition**:
    - We retrieved RNA-Seq data files from the Genomic Data Commons (GDC) API, specifically targeting the TCGA-BRCA project, which includes breast cancer samples.

2. **Data Cleaning**:
    - We removed rows with missing values in critical columns ('gene_name' and 'gene_type') and filtered out non-informative rows (e.g., 'N_unmapped', 'N_multimapping') to ensure the data quality.

#### Exploratory Data Analysis (EDA)

3. **Distribution Plots**:
    - **Histograms and Boxplots**: Initial visualizations of the untransformed data revealed a highly skewed distribution with many extreme values (outliers), especially in 'tpm_unstranded' and 'fpkm_unstranded'.
    - **Log-Transformed Distributions**: Applying a log transformation helped normalize the data, making the distributions more symmetrical and highlighting the differences more clearly.

4. **Principal Component Analysis (PCA)**:
    - PCA revealed significant variability in the data, with a small number of outliers (genes with extreme expression values). These outliers could represent biologically significant genes that merit further investigation.

#### Differential Expression Analysis

5. **Volcano Plot Analysis**:
    - **Volcano Plot**: The volcano plot is a crucial tool for visualizing the differential expression results. It displays the log2 fold change against the -log10 p-value, highlighting genes that are significantly upregulated or downregulated.
    - **Threshold Adjustment**: By adjusting the log2 fold change and p-value thresholds, we identified a subset of genes that are both statistically significant and biologically relevant.

6. **Significant Genes**:
    - The significant genes, identified based on the adjusted thresholds, are potential candidates for further study. These genes show substantial changes in expression and high statistical significance, indicating their potential involvement in cancer-related processes.

### Take-Home Messages

1. **Quality of Data**:
    - The RNA-Seq data retrieved from the TCGA-BRCA project required substantial preprocessing to handle missing values and non-informative rows. This step is crucial for ensuring the reliability of downstream analyses.

2. **Data Normalization**:
    - The log transformation of expression values was essential to normalize the data distribution, making it easier to identify meaningful patterns and outliers.

3. **Identifying Outliers**:
    - Outliers identified in the PCA and volcano plots suggest that certain genes exhibit extreme expression changes. These outliers could be key to understanding the mechanisms underlying breast cancer.

4. **Differential Expression Insights**:
    - The volcano plot provided a clear visual representation of differentially expressed genes. Adjusting the thresholds for log2 fold change and p-value helped pinpoint the most relevant genes for further study.

5. **Biological Relevance**:
    - The significant genes identified in this analysis could serve as biomarkers or therapeutic targets in breast cancer. Further validation and functional studies are needed to confirm their roles.

### Future Directions

To build upon this analysis, we recommend the following steps:

1. **Functional Annotation**:
    - Conduct functional annotation of the significant genes to understand their roles in biological pathways and processes related to cancer.

2. **Validation Studies**:
    - Perform experimental validation of the identified significant genes to confirm their involvement in breast cancer.

3. **Integrative Analysis**:
    - Combine RNA-Seq data with other omics data (e.g., proteomics, metabolomics) for a more comprehensive understanding of cancer biology.

4. **Clinical Correlation**:
    - Correlate the expression levels of significant genes with clinical outcomes to identify potential prognostic or predictive biomarkers.

This analysis demonstrates a robust approach to handling and analyzing high-throughput RNA-Seq data, providing valuable insights into the molecular mechanisms of breast cancer. By leveraging advanced data processing and visualization techniques, we can uncover critical information that contributes to our understanding of cancer and aids in the development of targeted therapies.



# Differential Gene Expression Analysis Using BERT for Interpretation

## Step 1: Differential Gene Expression Analysis

### Objective
Identify genes that are differentially expressed between cancerous and non-cancerous tissues.

### Data Preparation

In this section, we will prepare our data for differential gene expression analysis. This involves splitting the dataset into cancerous and non-cancerous samples, normalizing the expression values, and ensuring that the data is in a suitable format for analysis.

### Load the Data

First, we need to load the processed RNA-Seq data that we previously saved.

```python
import pandas as pd

# Load the processed RNA-Seq data
file_path = '/content/drive/MyDrive/Colab Notebooks/processed_rna_seq_data.csv'
all_rna_seq_df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to confirm it loaded correctly
print(all_rna_seq_df.head())


In [ ]:
import pandas as pd

# Load your RNA-seq data
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/processed_rna_seq_data.csv')

# Example of the first few rows of the dataset
print(data.head())


## Separate Cancerous and Non-Cancerous Samples
We will separate the data into cancerous and non-cancerous samples based on the submitter_id or any other available metadata that indicates the sample type.

In [ ]:
# Assuming 'submitter_id' indicates the sample type, for demonstration purposes
# In a real dataset, you would have a column that specifies whether each sample is cancerous or not

# Example separation (this needs to be adapted to your actual metadata)
# Here we assume that 'cancer' is part of the 'submitter_id' for cancerous samples
cancerous_samples = all_rna_seq_df[all_rna_seq_df['submitter_id'].str.contains('cancer', case=False, na=False)]
non_cancerous_samples = all_rna_seq_df[all_rna_seq_df['submitter_id'].str.contains('normal', case=False, na=False)]

# Display the number of samples in each group
print(f'Number of cancerous samples: {len(cancerous_samples)}')
print(f'Number of non-cancerous samples: {len(non_cancerous_samples)}')



In [ ]:
# Load the data
file_path = '/content/drive/MyDrive/Colab Notebooks/processed_rna_seq_data.csv'
all_rna_seq_df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to inspect metadata columns
print(all_rna_seq_df.head())

# Display the unique values in the 'submitter_id' column to understand the naming conventions
print(all_rna_seq_df['submitter_id'].unique())


In [ ]:
# Load the data
file_path = '/content/drive/MyDrive/Colab Notebooks/processed_rna_seq_data.csv'
all_rna_seq_df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to inspect metadata columns
print(all_rna_seq_df.head())

# Display the column names to understand available metadata
print(all_rna_seq_df.columns)


In [ ]:
import pandas as pd
import numpy as np

# Sample metadata to simulate the actual metadata
metadata = {
    'submitter_id': [
        'b36c0e8b-a9b1-46ac-8946-88e64a28ff5a', 'bd1f12ab-ee49-4e7e-aad4-14924c49d306',
        'd504a28c-f8e7-4c5e-b01a-23b4cdd1dd8a', 'f8efdeb0-a8f8-4e2d-a52e-43d033e4ce6f',
        'db01aba1-3a00-4bf9-9f24-8fba144144f3', 'c50bf088-8ad3-495c-b7b5-1eca9bb546ba'
    ],
    'sample_type': ['cancer', 'normal', 'cancer', 'normal', 'cancer', 'normal']
}

# Create a DataFrame from the sample metadata
metadata_df = pd.DataFrame(metadata)

# Display the sample metadata DataFrame
print(metadata_df)

# Load the processed RNA-Seq data
file_path = '/content/drive/MyDrive/Colab Notebooks/processed_rna_seq_data.csv'
all_rna_seq_df = pd.read_csv(file_path)

# Merge the RNA-Seq data with the metadata
merged_df = pd.merge(all_rna_seq_df, metadata_df, on='submitter_id')

# Display the first few rows of the merged DataFrame
print(merged_df.head())

# Separate the data into cancerous and non-cancerous samples based on 'sample_type' column
cancerous_samples = merged_df[merged_df['sample_type'] == 'cancer']
non_cancerous_samples = merged_df[merged_df['sample_type'] == 'normal']

# Display the number of samples in each group
print(f'Number of cancerous samples: {len(cancerous_samples)}')
print(f'Number of non-cancerous samples: {len(non_cancerous_samples)}')


In [ ]:
import pandas as pd
import numpy as np

# Sample metadata to simulate the actual metadata
metadata = {
    'submitter_id': [
        'b36c0e8b-a9b1-46ac-8946-88e64a28ff5a', 'bd1f12ab-ee49-4e7e-aad4-14924c49d306',
        'd504a28c-f8e7-4c5e-b01a-23b4cdd1dd8a', 'f8efdeb0-a8f8-4e2d-a52e-43d033e4ce6f',
        'db01aba1-3a00-4bf9-9f24-8fba144144f3', 'c50bf088-8ad3-495c-b7b5-1eca9bb546ba'
    ],
    'sample_type': ['cancer', 'normal', 'cancer', 'normal', 'cancer', 'normal']
}

# Create a DataFrame from the sample metadata
metadata_df = pd.DataFrame(metadata)

# Display the sample metadata DataFrame
print(metadata_df)

# Load the processed RNA-Seq data
file_path = '/content/drive/MyDrive/Colab Notebooks/processed_rna_seq_data.csv'
all_rna_seq_df = pd.read_csv(file_path)

# Merge the RNA-Seq data with the metadata
merged_df = pd.merge(all_rna_seq_df, metadata_df, on='submitter_id', how='left')

# Check for missing sample_type values
missing_sample_types = merged_df[merged_df['sample_type'].isna()]
print(f"Number of missing sample_type values: {len(missing_sample_types)}")

# Filter out the rows with missing sample_type
merged_df = merged_df.dropna(subset=['sample_type'])

# Display the first few rows of the merged DataFrame
print(merged_df.head())

# Separate the data into cancerous and non-cancerous samples based on 'sample_type' column
cancerous_samples = merged_df[merged_df['sample_type'] == 'cancer']
non_cancerous_samples = merged_df[merged_df['sample_type'] == 'normal']

# Display the number of samples in each group
print(f'Number of cancerous samples: {len(cancerous_samples)}')
print(f'Number of non-cancerous samples: {len(non_cancerous_samples)}')


In [ ]:
import pandas as pd
import numpy as np

# Sample metadata to simulate the actual metadata
metadata = {
    'submitter_id': [
        'b36c0e8b-a9b1-46ac-8946-88e64a28ff5a', 'bd1f12ab-ee49-4e7e-aad4-14924c49d306',
        'd504a28c-f8e7-4c5e-b01a-23b4cdd1dd8a', 'f8efdeb0-a8f8-4e2d-a52e-43d033e4ce6f',
        'db01aba1-3a00-4bf9-9f24-8fba144144f3', 'c50bf088-8ad3-495c-b7b5-1eca9bb546ba'
    ],
    'sample_type': ['cancer', 'normal', 'cancer', 'normal', 'cancer', 'normal']
}

# Create a DataFrame from the sample metadata
metadata_df = pd.DataFrame(metadata)

# Display the sample metadata DataFrame
print("Metadata DataFrame:")
print(metadata_df)

# Load the processed RNA-Seq data
file_path = '/content/drive/MyDrive/Colab Notebooks/processed_rna_seq_data.csv'
all_rna_seq_df = pd.read_csv(file_path)

# Inspect unique submitter_id values in the RNA-Seq data
unique_submitter_ids = all_rna_seq_df['submitter_id'].unique()
print(f"Unique submitter_ids in RNA-Seq data: {len(unique_submitter_ids)}")

# Merge the RNA-Seq data with the metadata
merged_df = pd.merge(all_rna_seq_df, metadata_df, on='submitter_id', how='left')

# Check for missing sample_type values
missing_sample_types = merged_df[merged_df['sample_type'].isna()]
print(f"Number of missing sample_type values: {len(missing_sample_types)}")

# Display the first few rows of the merged DataFrame with missing sample_type values
print("Rows with missing sample_type values:")
print(missing_sample_types.head())

# Filter out the rows with missing sample_type
merged_df = merged_df.dropna(subset=['sample_type'])

# Display the first few rows of the merged DataFrame
print("First few rows of the merged DataFrame:")
print(merged_df.head())

# Separate the data into cancerous and non-cancerous samples based on 'sample_type' column
cancerous_samples = merged_df[merged_df['sample_type'] == 'cancer']
non_cancerous_samples = merged_df[merged_df['sample_type'] == 'normal']

# Display the number of samples in each group
print(f'Number of cancerous samples: {len(cancerous_samples)}')
print(f'Number of non-cancerous samples: {len(non_cancerous_samples)}')


In [ ]:
expanded_metadata = {
    'submitter_id': [
        'b36c0e8b-a9b1-46ac-8946-88e64a28ff5a', 'bd1f12ab-ee49-4e7e-aad4-14924c49d306',
        'd504a28c-f8e7-4c5e-b01a-23b4cdd1dd8a', 'f8efdeb0-a8f8-4e2d-a52e-43d033e4ce6f',
        'db01aba1-3a00-4bf9-9f24-8fba144144f3', 'c50bf088-8ad3-495c-b7b5-1eca9bb546ba',
        # Add all unique submitter_ids from the RNA-Seq data here
    ],
    'sample_type': [
        'cancer', 'normal', 'cancer', 'normal', 'cancer', 'normal',
        # Corresponding sample types for each submitter_id
    ]
}

expanded_metadata_df = pd.DataFrame(expanded_metadata)

# Display the expanded metadata DataFrame
print("Expanded Metadata DataFrame:")
print(expanded_metadata_df)



In [ ]:
# Merge the RNA-Seq data with the expanded metadata
merged_df = pd.merge(all_rna_seq_df, expanded_metadata_df, on='submitter_id', how='left')

# Check for missing sample_type values
missing_sample_types = merged_df[merged_df['sample_type'].isna()]
print(f"Number of missing sample_type values: {len(missing_sample_types)}")

# Handle missing sample_type values by assigning a default value or further investigation
# For this example, we'll drop them
merged_df = merged_df.dropna(subset=['sample_type'])

# Display the first few rows of the merged DataFrame
print("First few rows of the merged DataFrame:")
print(merged_df.head())

# Separate the data into cancerous and non-cancerous samples based on 'sample_type' column
cancerous_samples = merged_df[merged_df['sample_type'] == 'cancer']
non_cancerous_samples = merged_df[merged_df['sample_type'] == 'normal']

# Display the number of samples in each group
print(f'Number of cancerous samples: {len(cancerous_samples)}')
print(f'Number of non-cancerous samples: {len(non_cancerous_samples)}')


In [ ]:
import os

# Check if the directory exists
directory_path = '/content/drive/MyDrive/Colab Notebooks/'
if not os.path.exists(directory_path):
    print(f"Directory does not exist: {directory_path}")

# List files in the directory
files_in_directory = os.listdir(directory_path)
print("Files in the directory:")
print(files_in_directory)


In [ ]:
import pandas as pd

# Sample data for the comprehensive metadata
sample_metadata = {
    "submitter_id": [
        'b36c0e8b-a9b1-46ac-8946-88e64a28ff5a',
        'bd1f12ab-ee49-4e7e-aad4-14924c49d306',
        'd504a28c-f8e7-4c5e-b01a-23b4cdd1dd8a',
        'f8efdeb0-a8f8-4e2d-a52e-43d033e4ce6f',
        'db01aba1-3a00-4bf9-9f24-8fba144144f3',
        'c50bf088-8ad3-495c-b7b5-1eca9bb546ba'
        # Add more submitter_ids as needed
    ],
    "sample_type": [
        'cancer', 'normal', 'cancer', 'normal', 'cancer', 'normal'
        # Add corresponding sample types as needed
    ]
}

# Create a DataFrame
comprehensive_metadata_df = pd.DataFrame(sample_metadata)

# Define the path to save the file
comprehensive_metadata_path = '/content/drive/MyDrive/Colab Notebooks/comprehensive_metadata.csv'

# Save the DataFrame to a CSV file
comprehensive_metadata_df.to_csv(comprehensive_metadata_path, index=False)

print(f"Comprehensive metadata file created at {comprehensive_metadata_path}")


In [ ]:
import pandas as pd

# Define the path to the comprehensive metadata file
comprehensive_metadata_path = '/content/drive/MyDrive/Colab Notebooks/comprehensive_metadata.csv'

# Load the comprehensive metadata
comprehensive_metadata_df = pd.read_csv(comprehensive_metadata_path)

# Display the first few rows of the comprehensive metadata
print("Comprehensive Metadata DataFrame:")
print(comprehensive_metadata_df.head())


In [ ]:
# Merging the RNA-Seq data with the comprehensive metadata
merged_df = pd.merge(all_rna_seq_df, comprehensive_metadata_df, on='submitter_id', how='left')

# Check for missing sample_type values
missing_sample_types = merged_df[merged_df['sample_type'].isna()]
print(f"Number of missing sample_type values after comprehensive merge: {len(missing_sample_types)}")

# Drop rows with missing sample_type values for further analysis
cleaned_df = merged_df.dropna(subset=['sample_type'])

# Display the first few rows of the cleaned merged DataFrame
print("First few rows of the cleaned merged DataFrame:")
print(cleaned_df.head())


In [ ]:
# Separate the data into cancerous and non-cancerous samples based on 'sample_type' column
cancerous_samples = cleaned_df[cleaned_df['sample_type'] == 'cancer']
non_cancerous_samples = cleaned_df[cleaned_df['sample_type'] == 'normal']

# Display the number of samples in each group
print(f'Number of cancerous samples: {len(cancerous_samples)}')
print(f'Number of non-cancerous samples: {len(non_cancerous_samples)}')


In [ ]:
import numpy as np
from scipy.stats import ttest_ind

# Calculate log2 fold change and p-values
def calculate_differential_expression(cancer_df, normal_df, gene_column='gene_id', expression_column='tpm_unstranded'):
    diff_exp_results = {
        'gene_id': [],
        'log2_fold_change': [],
        'p_value': []
    }

    unique_genes = cancer_df[gene_column].unique()

    for gene in unique_genes:
        cancer_expression = cancer_df[cancer_df[gene_column] == gene][expression_column]
        normal_expression = normal_df[normal_df[gene_column] == gene][expression_column]

        if len(cancer_expression) > 1 and len(normal_expression) > 1:
            log2_fc = np.log2(cancer_expression.mean() / normal_expression.mean())
            _, p_value = ttest_ind(cancer_expression, normal_expression)

            diff_exp_results['gene_id'].append(gene)
            diff_exp_results['log2_fold_change'].append(log2_fc)
            diff_exp_results['p_value'].append(p_value)

    return pd.DataFrame(diff_exp_results)

# Perform differential expression analysis
diff_exp_df = calculate_differential_expression(cancerous_samples, non_cancerous_samples)

# Display the first few rows of the differential expression results
print("Differential Expression Results:")
print(diff_exp_df.head())
